In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
cur_path = os.getcwd()

data = []
labels = []

data_folder = 'Drowsiness_dataset'
path = os.path.join(cur_path,data_folder)
train = os.listdir(path)

In [3]:
for label in train :
    if "yawn" in label :
        file_path = os.path.join(path,label)
        files = os.listdir(file_path)
        for img in files :
            obj_path = os.path.join(file_path,img)
            image = cv2.imread(obj_path)
            width = int(image.shape[1] * 0.5)
            height = int(image.shape[0] * 0.5)
            dim = (width, height)
            resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
            gray = cv2.equalizeHist(gray)
            data.append(np.array(gray))
            labels.append(label)

In [4]:
data = np.array(data)
labels = np.array(labels)

print(data.shape)
print(labels.shape)

(1448, 240, 320)
(1448,)


In [5]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [6]:
data = np.expand_dims(data,axis=3)
label_encoding = LabelBinarizer()
labels = label_encoding.fit_transform(labels)

X_train, X_val, y_train, y_val = train_test_split(data, labels,shuffle=True, stratify=labels, test_size=0.25, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)

(1086, 240, 320, 1) (362, 240, 320, 1) (1086, 1) (362, 1)


In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import F1Score

In [11]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.25))

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(SpatialDropout2D(0.25))

model.add(Flatten())
model.add(Dense(48, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(24, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 238, 318, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 119, 159, 32)      0         
 g2D)                                                            
                                                                 
 spatial_dropout2d_2 (Spati  (None, 119, 159, 32)      0         
 alDropout2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 115, 155, 16)      12816     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 57, 77, 16)        0         
 g2D)                                                            
                                                      

In [12]:
earlystop = EarlyStopping(monitor='accuracy',mode='max', patience=3,start_from_epoch=8)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
epochs = 64
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val), callbacks=[earlystop])

Epoch 1/64
34/34 [==============================] - 21s 621ms/step - loss: 0.1665 - accuracy: 0.9199 - val_loss: 0.2695 - val_accuracy: 0.9171
Epoch 2/64
34/34 [==============================] - 21s 618ms/step - loss: 0.1210 - accuracy: 0.9383 - val_loss: 0.2941 - val_accuracy: 0.9144
Epoch 3/64
34/34 [==============================] - 21s 610ms/step - loss: 0.1544 - accuracy: 0.9300 - val_loss: 0.3134 - val_accuracy: 0.9088
Epoch 4/64
34/34 [==============================] - 21s 615ms/step - loss: 0.1712 - accuracy: 0.9273 - val_loss: 0.2902 - val_accuracy: 0.9171
Epoch 5/64
34/34 [==============================] - 21s 621ms/step - loss: 0.1402 - accuracy: 0.9328 - val_loss: 0.3223 - val_accuracy: 0.9061
Epoch 6/64
34/34 [==============================] - 21s 622ms/step - loss: 0.0990 - accuracy: 0.9576 - val_loss: 0.3664 - val_accuracy: 0.9061
Epoch 7/64
34/34 [==============================] - 21s 629ms/step - loss: 0.1424 - accuracy: 0.9457 - val_loss: 0.2801 - val_accuracy: 0.9088

In [15]:
history_2 = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_val, y_val), callbacks=[earlystop])

Epoch 1/64
17/17 [==============================] - 21s 1s/step - loss: 0.1133 - accuracy: 0.9475 - val_loss: 0.3258 - val_accuracy: 0.9227
Epoch 2/64
17/17 [==============================] - 20s 1s/step - loss: 0.1036 - accuracy: 0.9558 - val_loss: 0.3313 - val_accuracy: 0.9171
Epoch 3/64
17/17 [==============================] - 20s 1s/step - loss: 0.1018 - accuracy: 0.9530 - val_loss: 0.3636 - val_accuracy: 0.9116
Epoch 4/64
17/17 [==============================] - 20s 1s/step - loss: 0.1203 - accuracy: 0.9466 - val_loss: 0.3572 - val_accuracy: 0.9116
Epoch 5/64
17/17 [==============================] - 20s 1s/step - loss: 0.0991 - accuracy: 0.9604 - val_loss: 0.3844 - val_accuracy: 0.9144
Epoch 6/64
17/17 [==============================] - 20s 1s/step - loss: 0.0939 - accuracy: 0.9669 - val_loss: 0.4191 - val_accuracy: 0.9171
Epoch 7/64
17/17 [==============================] - 20s 1s/step - loss: 0.0742 - accuracy: 0.9715 - val_loss: 0.4229 - val_accuracy: 0.9254
Epoch 8/64
17/17 [==

In [18]:
Model_file_name = "model.keras"
sav_dir = os.path.join(cur_path,Model_file_name)
model.save(sav_dir)